<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/basics/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ベイズの定理を使ったスパムフィルタリング

## ベイズの定理
ベイズの定理は次のように表されます：

$ P(H|D) = \frac{P(D|H) \cdot P(H)}{P(D)} $

ここで、
- $ P(H|D) $ は事後確率（データ $ D $ が観測された後の仮説 $ H $ の確率）
- $ P(D|H) $ は尤度（仮説 $ H $ が真である場合にデータ $ D $ が観測される確率）
- $ P(H) $ は事前確率（データ $ D $ を観測する前の仮説 $ H $ の確率）
- $ P(D) $ は周辺確率（データ $ D $ の全体的な観測確率）

## スパムフィルタにおけるベイズの定理の適用
スパムフィルタリングにおいて、次のように解釈します：
- 仮説 $ H $：メールがスパムであること
- データ $ D $：メールに特定の単語が含まれていること

これを用いて、メールがスパムである確率 $ P(\text{spam}|\text{email}) $ を計算します。

----

## ステップ1: 教師データ準備
まず、スパムと非スパムのメールのデータセットを準備します。新しいメールが届いたときには、このデータセットをもとにしてスパムか非スパムかを判定します。


In [ ]:
import pandas as pd

# サンプルデータセットの作成
data = {
    'email': [
        'Win a million dollars now',   # スパム
        'Lowest prices in pharmacy',    # スパム
        'Hello, how are you?',          # 非スパム
        'Meeting tomorrow at 10am',     # 非スパム
        'Congratulations, you won a prize', # スパム
        'Get cheap meds online',        # スパム
        'Can we reschedule our appointment?', # 非スパム
        'Your account has been hacked'  # スパム
    ],
    'label': ['spam', 'spam', 'ham', 'ham', 'spam', 'spam', 'ham', 'spam']
}

df = pd.DataFrame(data)
df


,email,label
0,Win a million dollars now,spam
1,Lowest prices in pharmacy,spam
2,"Hello, how are you?",ham
3,Meeting tomorrow at 10am,ham
4,"Congratulations, you won a prize",spam
5,Get cheap meds online,spam
6,Can we reschedule our appointment?,ham
7,Your account has been hacked,spam


## ステップ2: データの前処理

次に、メールのテキストを前処理し、単語の出現回数をカウントします。

各行は１通のメールに相当します。横軸の大半（label列を除く）は単語で、存在回数を表します。label列は、そのメールがスパムか非スパムかを表します。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 単語のカウントを行うためのベクトライザを作成
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['email'])

# 特徴量（単語）の名前を取得
feature_names = vectorizer.get_feature_names_out()

# 単語の出現回数をデータフレームに変換
word_count_df = pd.DataFrame(X.toarray(), columns=feature_names)
word_count_df['label'] = df['label']
word_count_df


,10am,account,appointment,are,at,been,can,cheap,congratulations,dollars,...,prices,prize,reschedule,tomorrow,we,win,won,you,your,label
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,spam
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,spam
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,ham
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,ham
4,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,1,0,spam
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,spam
6,0,0,1,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,ham
7,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,spam


## ステップ3: ベイズの定理の適用

ベイズの定理を使って、各メールがスパムかどうかを予測します。まず、各単語の条件付き確率を計算します。

In [ ]:
import numpy as np

# スパムと非スパムの頻度を計算
spam_emails = word_count_df[word_count_df['label'] == 'spam']
ham_emails = word_count_df[word_count_df['label'] == 'ham']

# 各単語のスパムおよび非スパムの確率を計算
spam_word_probs = (spam_emails.drop('label', axis=1).sum() + 1) / (spam_emails.drop('label', axis=1).sum().sum() + len(feature_names))
ham_word_probs = (ham_emails.drop('label', axis=1).sum() + 1) / (ham_emails.drop('label', axis=1).sum().sum() + len(feature_names))

# 例示用。スパムらしい単語とスパムらしくない単語を挙げています
selected_words = ['win', 'prize', 'pharmacy', 'meeting', 'reschedule']
for word in selected_words:
    spam_count = spam_emails[word].sum()
    ham_count = ham_emails[word].sum()
    spam_prob = spam_word_probs[word]
    ham_prob = ham_word_probs[word]

    print(f"Word: '{word}'")
    print(f"  Spam count: {spam_count}, Ham count: {ham_count}")
    print(f"  P(word|spam): {spam_prob:.4f}, P(word|ham): {ham_prob:.4f}")

Word: 'win'
  Spam count: 1, Ham count: 0
  P(word|spam): 0.0370, P(word|ham): 0.0217
Word: 'prize'
  Spam count: 1, Ham count: 0
  P(word|spam): 0.0370, P(word|ham): 0.0217
Word: 'pharmacy'
  Spam count: 1, Ham count: 0
  P(word|spam): 0.0370, P(word|ham): 0.0217
Word: 'meeting'
  Spam count: 0, Ham count: 1
  P(word|spam): 0.0185, P(word|ham): 0.0435
Word: 'reschedule'
  Spam count: 0, Ham count: 1
  P(word|spam): 0.0185, P(word|ham): 0.0435


#### 事前確率の計算

事前確率 $P(\text{spam})$ と $P(\text{ham})$ は、スパムメールおよび非スパムメールの全体に対する割合です。

$$ P(\text{spam}) = \frac{\text{スパムメールの数}}{\text{全メールの数}} $$
$$ P(\text{ham}) = \frac{\text{非スパムメールの数}}{\text{全メールの数}} $$

In [ ]:
P_spam = len(spam_emails) / len(word_count_df)
P_ham = len(ham_emails) / len(word_count_df)
print("スパムの事前確率　", P_spam)
print("非スパムの事前確率", P_ham)

スパムの事前確率　 0.625
非スパムの事前確率 0.375



#### 事前確率の対数の初期化

初期値として事前確率の対数を設定します。対数を取ることで計算の安定性を保ちます。

$$ \log P(\text{spam})$$
$$ \log P(\text{ham}) $$

#### 尤度の累積計算

メール内の各単語について、スパムメールおよび非スパムメールにその単語が含まれる確率（尤度）を累積して計算します。

各単語 $w_i$ について、尤度 $P(w_i|\text{spam})$ および $P(w_i|\text{ham})$ を累積します。

$$ \log P(\text{email}|\text{spam}) = \sum_{i} \log P(w_i|\text{spam}) $$
$$ \log P(\text{email}|\text{ham}) = \sum_{i} \log P(w_i|\text{ham}) $$

##### 対数を取る理由

対数を取ることで、確率の掛け算を足し算に変えることができます。確率の積は非常に小さくなることが多いため、一般的に行われます。

$$ P(w_1|\text{spam}) \cdot P(w_2|\text{spam}) \cdot \ldots \cdot P(w_n|\text{spam}) $$
は、
$$ \log P(w_1|\text{spam}) + \log P(w_2|\text{spam}) + \ldots + \log P(w_n|\text{spam}) $$
に変換されます。

#### 事後確率の比較

累積した確率の対数を比較し、スパム確率が非スパム確率より高ければ「spam」、そうでなければ「ham」を返します。

$$ P(\text{spam}|\text{email}) > P(\text{ham}|\text{email}) \Rightarrow \text{spam} $$
$$ P(\text{spam}|\text{email}) \leq P(\text{ham}|\text{email}) \Rightarrow \text{ham} $$

この関数では、ベイズの定理に基づいて新しいメールがスパムである確率を計算し、スパムか非スパムかを予測します。

In [ ]:
# 新しいメールのスパム確率を計算する関数
def predict_spam(email):

    # 事前確率の計算
    P_spam = len(spam_emails) / len(word_count_df)
    P_ham  = len(ham_emails) / len(word_count_df)

    # 対数をとる
    # 対数の底はネイピア数 e (約 2.7) ですが、気にしなくて結構です
    log_P_spam = np.log(P_spam)
    log_P_ham  = np.log(P_ham)

    print("　スパムの事前確率", f"{np.exp(log_P_spam):.8f}", f"（対数 {log_P_spam:.4f}）")
    print("非スパムの事前確率", f"{np.exp(log_P_ham):.8f}", f"（対数 {log_P_ham:.4f}）")
    print("...")

    # ベイズの定理に従って更新する
    words = email.split()
    for word in words:
        if word in spam_word_probs:
            log_P_spam += np.log(spam_word_probs[word])
            print(f"   単語 {word} -->    スパムの確率更新 ：対数尤度 {np.log(spam_word_probs[word]):.4f} を加算")
        if word in ham_word_probs:
            log_P_ham += np.log(ham_word_probs[word])
            print(f"   単語 {word} -->  非スパムの確率更新 ：対数尤度 {np.log(ham_word_probs[word]):.4f} を加算")

    print(f"　このメールがスパムである事後確率 {np.exp(log_P_spam):.8f}", f"（対数 {log_P_spam:.4f}）")
    print(f"このメールが非スパムである事後確率 {np.exp(log_P_ham):.8f}", f"（対数 {log_P_ham:.4f}）")

    if log_P_spam > log_P_ham:
      return 'スパム'
    else:
      return '非スパム'

# テスト用のメール
test_email = 'Win a free prize now'
print(f'→メール "{test_email}" は {predict_spam(test_email)} と判別されました')


　スパムの事前確率 0.62500000 （対数 -0.4700）
非スパムの事前確率 0.37500000 （対数 -0.9808）
...
   単語 prize -->    スパムの確率更新 ：対数尤度 -3.2958 を加算
   単語 prize -->  非スパムの確率更新 ：対数尤度 -3.8286 を加算
   単語 now -->    スパムの確率更新 ：対数尤度 -3.2958 を加算
   単語 now -->  非スパムの確率更新 ：対数尤度 -3.8286 を加算
　このメールがスパムである事後確率 0.00085734 （対数 -7.0617）
このメールが非スパムである事後確率 0.00017722 （対数 -8.6381）
→メール "Win a free prize now" は スパム と判別されました
